In [1]:
# qiskit circuit & machine
from qiskit import QuantumCircuit
import qiskit_aer
from qiskit_ibm_runtime.fake_provider import FakeVigo
from qiskit.circuit.library import QFT
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# widget-related
import anywidget
import traitlets

# widget function
from state_uncertainty_vis import drawStateUncertainty, uncertainty_design
import numpy as np
import pandas as pd

# host setting for dev mode
_SS_ = "http://localhost:5175/"

In [2]:
class UncertaintyVis(anywidget.AnyWidget):
    _esm = _SS_ + "index.js?anywidget"
    vis_data = traitlets.Unicode().tag(sync=True)
    design = traitlets.Unicode().tag(sync=True)
    mode = traitlets.Unicode("uncertainty").tag(sync=True)

def getUncertaintyVis(job, transpiled, design={}, bootstrap=False, sampling_counts=10000, sample_size=None):
    prep = drawStateUncertainty(job, transpiled, bootstrap=bootstrap, sampling_counts=sampling_counts, sample_size=sample_size)
    vis_data = prep.toJSON()
    _design = uncertainty_design(design)
    __design = _design.toJSON()
    viewer = UncertaintyVis(vis_data=vis_data, design=__design)
    return (viewer, prep, _design)

In [12]:
qft = QFT(3).to_gate()

qcb = QuantumCircuit(3, 3)
qcb.h([0, 1, 2])
qcb.append(qft, [0, 1, 2])
qcb.measure_all()

optimization_level = 1
backend = qiskit_aer.AerSimulator.from_backend(FakeVigo())
pass_manager = generate_preset_pass_manager(backend=backend, optimization_level=optimization_level, seed_transpiler=1)

qcb_trans = pass_manager.run(qcb)
job = backend.run(qcb_trans, shots=1000)

In [13]:
(unc, dt, design) = getUncertaintyVis(job, qcb_trans, design={"mark": "bar", "direction": "vertical"}, sampling_counts=100)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [14]:
unc

UncertaintyVis(design='{"mark": "bar", "direction": "vertical"}', vis_data='{"data": {"101 000": {"measure": 3…

'{"mark": "bar", "direction": "vertical"}'

In [11]:
dt.toJSON()

'{"data": {"001 000": {"measure": 18, "prob_measure": 0.018, "state": "001 000", "mean": 42.9885, "std": 5.367827097625258, "interval": [33.0, 54.0], "lb": 33.0, "ub": 54.0, "prob_mean": 0.042988500000000006, "prob_std": 0.005367827097625258, "prob_interval": [0.033, 0.054], "prob_lb": 0.033, "prob_ub": 0.054}, "101 000": {"measure": 9, "prob_measure": 0.009, "state": "101 000", "mean": 21.5122, "std": 3.8304896762685576, "interval": [14.0, 29.0], "lb": 14.0, "ub": 29.0, "prob_mean": 0.021512200000000002, "prob_std": 0.0038304896762685575, "prob_interval": [0.014, 0.029], "prob_lb": 0.014, "prob_ub": 0.029}, "100 000": {"measure": 22, "prob_measure": 0.022, "state": "100 000", "mean": 52.428, "std": 5.876428847522957, "interval": [41.0, 64.0], "lb": 41.0, "ub": 64.0, "prob_mean": 0.05242800000000002, "prob_std": 0.005876428847522958, "prob_interval": [0.041, 0.064], "prob_lb": 0.041, "prob_ub": 0.064}, "110 000": {"measure": 13, "prob_measure": 0.013, "state": "110 000", "mean": 31.091

In [7]:
dt.draw()

alt.LayerChart(...)

In [21]:
import numpy as np
prob = 0.7
states = np.array(["a", "b", "c"])
value = "a"
probs = np.repeat( (1-prob)/(len(states) - 1), len(states))
this = np.argwhere(states == value)
print(this)
probs[this] = prob
probs
np.random.choice(states, 1, p=probs)

[[0]]


array(['a'], dtype='<U1')

In [7]:
import numpy as np
np.argwhere( np.array(["a", "b", "c"]) == "a")

array([[0]])

In [9]:
import numpy as np
resample_collection = np.array([1, 2, 3])
piecewise_func = [lambda x: x * 2 for x in [1, 2, 3]]
np.piecewise(resample_collection, 
           [resample_collection == x for x in [1, 2, 3]], 
           [lambda x: f(x) for f in piecewise_func])

array([2, 4, 6])

In [11]:
import numpy as np
def gen_alt_output(value, prob, states):
    probs = np.repeat( (1-prob)/(len(states) - 1), len(states))
    this = np.argwhere(states == value)
    probs[this] = prob
    print(probs)
    def o():
        return np.random.choice(states, 1, p=probs)
    return o

In [34]:
states = np.array(['00010 00000',  '01010 00000'])

In [59]:
error_rates = np.array([0, 0.0005020255558466025, 0, 0, 0.006589629429362032, 0, 0.006589629429362032, 0, 0.0005020255558466025, 0, 0, 0, 0.0006149355812506126, 0, 0.0092971260000492, 0.0092971260000492, 0.0092971260000492, 0.006589629429362032, 0, 0.006589629429362032, 0, 0, 0.0092971260000492, 0, 0.0092971260000492, 0, 0.0005020255558466025, 0, 0.006589629429362032, 0.006589629429362032, 0.006589629429362032, 0, 0.022499999999999964, 0.021500000000000075, 0.014599999999999946])
states = np.array(['111 000', '001 000', '110 000', '100 000', '101 000', '011 000', '010 000', '000 000'])

def simulate_error_process(error_rates, states):
    def error_process(state):
        proc = pd.Series(error_rates)
        proc = proc.apply(lambda x: np.random.binomial(1, 1-x, 1)[0]).to_numpy()
        # print(proc)
        if proc.sum() == proc.size:
            return state
        else:
            pv = 1 / (len(states) - 1)
            probs = np.repeat(pv, len(states))
            x = np.argwhere(states == state)
            probs[x] = 0
            return np.random.choice(states, 1, p = probs)[0]
    return error_process
        
func = simulate_error_process(error_rates, states)
results = []
for i in range(10000):
    results.append(func(states[0]))
results = np.array(results)

In [60]:
np.unique(results, return_counts=True)

(array(['000 000', '001 000', '010 000', '011 000', '100 000', '101 000',
        '110 000', '111 000'], dtype='<U7'),
 array([ 194,  164,  195,  196,  207,  219,  183, 8642]))